In [12]:
import requests 
import re as regex
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import aries_cloudagent.wallet.util as util
import base64
import base58
import uuid

agent1 = {}
agent2 = {}
plugin = {}

## Enter Invite Url:

In [13]:
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMjI5MGI1ODctN2UyMi00NGQ0LTk1N2QtNTZlNGUxZGE2YzE1IiwgInJlY2lwaWVudEtleXMiOiBbIkd6R1pyVUdtTlJ3YlpLd1RkTTg3NzF0Rmp5Y3AzSmFYZ0Zxb2pqWjhrSnlWIl0sICJsYWJlbCI6ICJNYWluIChhZG1pbikiLCAic2VydmljZUVuZHBvaW50IjogImh0dHA6Ly9hZ2VudDEubG9jYWxob3N0In0=


In [14]:
# Process invite url, delete white spaces
agent1['url'] = agent1['url'].replace(" ", "")
# Regex(substitution) to extract only the invite string from url
agent1['invite_string_b64'] = regex.sub(
           r".*(c\_i\=)", 
           r"", 
           agent1['url'])
# Decoding invite string using base64 decoder
agent1['invite_string'] = base64.b64decode(agent1['invite_string_b64'])
# Converting our invite json string into a dictionary 
agent1['invite'] = json.loads(agent1['invite_string'])

print("Decoded invitation:\n")
pp.pprint(agent1['invite'])

Decoded invitation:

{'@id': '2290b587-7e22-44d4-957d-56e4e1da6c15',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Main (admin)',
 'recipientKeys': ['GzGZrUGmNRwbZKwTdM8771tFjycp3JaXgFqojjZ8kJyV'],
 'serviceEndpoint': 'http://agent1.localhost'}


In [15]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

# print(base58.b58encode(did))
# print(plugin['did'])
print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 4JZwLwuCwjwZaEhbHPSbPXXkdkQonDhasBju6PjVLyC4GR7yPM8GN81qqphYBKJ5VK7PKARrAp8WmxPqNB81Xt6P

Public key: 
 aueVNDZP3U32oscmVreXwZmE8Di2iwZKUh5ZzsXahdd


**Sending a connection request to acapy**

In [16]:
uniqueId = str(uuid.uuid4())
# print(uniqueId)

# our request body
message = {
        "@id":  uniqueId,
        "~transport": {
          "return_route": "all"
        },
        "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/request",
        "label": "Plugin",
        "connection": {
          "DID": plugin['did'],
          "DIDDoc": {
            "@context": "https://w3id.org/did/v1",
            "id": plugin['did'],
            "publicKey": [{
              "id": plugin['did'] + "#keys-1",
              "type": "Ed25519VerificationKey2018",
              "controller": plugin['did'],
              "publicKeyBase58": plugin['public_key_b58']
            }],
            "service": [{
              "id": plugin['did'] + ";indy",
              "type": "IndyAgent",
              "recipientKeys": plugin['public_key_b58'],
              "serviceEndpoint": ""
            }]
          }
        }
      }

**Encoding/packing the request**

In [17]:
# encoding it with aries crypto function using the key that was
# given to us by aca-py in recipientKeys
decodedAcapyKey = base58.b58decode(agent1['invite']['recipientKeys'][0])
ourPrivateKey = plugin['keypair'][1]
# print(plugin['did'])
encodedMessage = \
    crypto.encode_pack_message(json.dumps(message), [decodedAcapyKey], ourPrivateKey)

encodedMessage = encodedMessage.decode("ascii")
# print("Encoded message: \n")
# pp.pprint(encodedMessage)

In [18]:
connectionRequestResponse = requests.post(agent1['invite']['serviceEndpoint'], data=encodedMessage)
assert connectionRequestResponse.text != "", "invalid response from acapy"

**Response from aca-py**

In [19]:
# a bit of a hack to simplify message unpacking,
# decode_pack_message needs a callable object for some reason
def unpackMessage(message, privateKey):
    class FindKey:
        def __init__(self, key):
            self.key = key

        def __call__(self, argument):
            return self.key

    find_key = FindKey(privateKey)   
    return crypto.decode_pack_message(message, find_key)
    
connectionRequestResponseUnpacked = \
    unpackMessage(connectionRequestResponse.text, plugin['keypair'][1])

connectionRequestResponseDict = json.loads(connectionRequestResponseUnpacked[0])

print("Decoded acapy response: \n")
pp.pprint(connectionRequestResponseUnpacked)

Decoded acapy response: 

('{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/response", '
 '"@id": "b5c86c9d-92b4-4e71-9ac2-1dff3243b483", "~thread": {"thid": '
 '"49d13a33-ff13-4163-b9c9-e17537dfdc58"}, "connection~sig": {"@type": '
 '"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/signature/1.0/ed25519Sha512_single", '
 '"signature": '
 '"EnBv727-7UG_cSwRdlMchATziZ5wfF4NWNTnhCDYL0BrDAkeEuJvzHTOF38cCgZmRmcKzNwtA25uQF9WVUUaAQ==", '
 '"sig_data": '
 '"AAAAAF69J2t7IkRJRCI6ICJXS1htS0FHanFtUHlxN0x6amNqb3F6IiwgIkRJRERvYyI6IHsiQGNvbnRleHQiOiAiaHR0cHM6Ly93M2lkLm9yZy9kaWQvdjEiLCAiaWQiOiAiZGlkOnNvdjpXS1htS0FHanFtUHlxN0x6amNqb3F6IiwgInB1YmxpY0tleSI6IFt7ImlkIjogImRpZDpzb3Y6V0tYbUtBR2pxbVB5cTdMempjam9xeiMxIiwgInR5cGUiOiAiRWQyNTUxOVZlcmlmaWNhdGlvbktleTIwMTgiLCAiY29udHJvbGxlciI6ICJkaWQ6c292OldLWG1LQUdqcW1QeXE3THpqY2pvcXoiLCAicHVibGljS2V5QmFzZTU4IjogIkd5cjd0Y3llQlFSTnoxQnNFWUR0b1FNR0FIendYbWlmODkxZVo3RHp3aE13In1dLCAiYXV0aGVudGljYXRpb24iOiBbeyJ0eXBlIjogIkVkMjU1MTlTaWduYXR1cmVBdXRoZW50aWNhdGlvbjIwMT

**Unpacking connection data embedded in the response**

In [20]:
sig_data_raw = connectionRequestResponseDict['connection~sig']['sig_data']
#  (this is a hack)replacing artifacts that sometimes happen
sig_data_raw = sig_data_raw.replace("-", "1")
sig_data_raw = sig_data_raw.replace("_", "1")

sig_data_raw = base64.b64decode(sig_data_raw)
# avoid first 8 characters as they are a time signature
sig_data_raw = sig_data_raw[8:]
sig_data_raw = sig_data_raw.decode('ascii')

sig_data = json.loads(sig_data_raw)
pp.pprint(sig_data)

{'DID': 'WKXmKAGjqmPyq7Lzjcjoqz',
 'DIDDoc': {'@context': 'https://w3id.org/did/v1',
            'authentication': [{'publicKey': 'did:sov:WKXmKAGjqmPyq7Lzjcjoqz#1',
                                'type': 'Ed25519SignatureAuthentication2018'}],
            'id': 'did:sov:WKXmKAGjqmPyq7Lzjcjoqz',
            'publicKey': [{'controller': 'did:sov:WKXmKAGjqmPyq7Lzjcjoqz',
                           'id': 'did:sov:WKXmKAGjqmPyq7Lzjcjoqz#1',
                           'publicKeyBase58': 'Gyr7tcyeBQRNz1BsEYDtoQMGAHzwXmif891eZ7DzwhMw',
                           'type': 'Ed25519VerificationKey2018'}],
            'service': [{'id': 'did:sov:WKXmKAGjqmPyq7Lzjcjoqz;indy',
                         'priority': 0,
                         'recipientKeys': ['Gyr7tcyeBQRNz1BsEYDtoQMGAHzwXmif891eZ7DzwhMw'],
                         'serviceEndpoint': 'http://agent1.localhost',
                         'type': 'IndyAgent'}]}}


**Adding the connection to connection list**

In [21]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": sig_data['DIDDoc'],
        "myKey": plugin['keypair'][1]
    }
]

def packMessage(message, connection):
    # pass in our private key and recipient key to the encode_pack_message
    decodedRecipientKey = base58.b58decode(connection['DIDDoc']['service'][0]['recipientKeys'][0])
    packedMessage = \
    crypto.encode_pack_message(json.dumps(message), \
                               [decodedRecipientKey], \
                               connection['myKey'])
    return packedMessage.decode('ascii')


pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:WKXmKAGjqmPyq7Lzjcjoqz#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:WKXmKAGjqmPyq7Lzjcjoqz',
             'publicKey': [{'controller': 'did:sov:WKXmKAGjqmPyq7Lzjcjoqz',
                            'id': 'did:sov:WKXmKAGjqmPyq7Lzjcjoqz#1',
                            'publicKeyBase58': 'Gyr7tcyeBQRNz1BsEYDtoQMGAHzwXmif891eZ7DzwhMw',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:WKXmKAGjqmPyq7Lzjcjoqz;indy',
                          'priority': 0,
                          'recipientKeys': ['Gyr7tcyeBQRNz1BsEYDtoQMGAHzwXmif891eZ7DzwhMw'],
                          'serviceEndpoint': 'http://agent1.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Main (admin)',
  'myKey': b'\xa53l\x02\xef@\xf4snzf\xbeb\xb9\x19\xa2\xf8\x8f\xb5H\xa

### FEATURE DISCOVERY

In [22]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
message['query'] = '*'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'c6dd21cb-abe2-4f27-b41a-b53a45200bf6',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'https://didcomm.org/notification/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/basicmessage/1.0'},
               {'pid': 'https://didcomm.org/present-proof/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-holder/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/credential-presentation/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/credential-issuance/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-credential-definitions/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0'},
               {'pid': 'https://didcomm.org/trust_ping/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0'},
   

## Dids

In [23]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-list-dids"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '7fe53168-2b92-40cc-9802-eebb03baede8',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/list-dids',
 'result': [{'did': 'WKXmKAGjqmPyq7Lzjcjoqz',
             'verkey': 'Gyr7tcyeBQRNz1BsEYDtoQMGAHzwXmif891eZ7DzwhMw'}],
 '~thread': {'thid': '2642f320-428b-4a03-ae80-ee1c87015b97'}}


## Public did

In [24]:
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-public-did"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'f285d57c-2f66-4854-aa0c-c03abce156e9',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '~thread': {'thid': '06031017-63dc-4a01-9b2f-3b105690fe59'}}


In [25]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/query"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
message['comment'] = 'qweqwe'

# message['comment'] = "memes"
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '81397632-e925-4be9-84e8-6eddaa18df99',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/query',
 'comment': 'qweqwe',
 '~thread': {'thid': '74836db2-85f7-492f-9708-076b876d9690'}}
